# Day4 「Pythonテトリス」の作り方
テトリスは、ミノが置かれていない状態を「0」、I字型のミノを「1」、L字型のミノを「2」・・・と数字で呼ぶことにして、<br>
`0`から`7`までの数字が`22`段 × `10`列 = `220`個並んだ「NumPy配列」によって表現されている、とDay2で述べました。<br><br>

テトリスのコードの仕組みを把握するポイントは、
<b>

1. ミノの形・回転・移動・落下というルールを、「NumPy配列」によってどうやって表現できる？
1. ミノをどの向きでどこに落とすか、どうやって決めればいい？

</b>
という2点です。<br>
文法学習が終わって中盤戦に入るDay4では、本番で使われるコードでこれらの実装を確認し、実際に挙動を試してみることでその理解を深めます。<br>

## 0. チュートリアルのゴール
<b>ルールベースの場合には`block_controller.py`を、AIの場合には`block_controller_train.py`を書き換えられる</b>ようになることが、学習のゴールです。

### 文法学習の補足
Day3までの文法学習では、1つの`.py`ファイルや`.ipynb`ファイルでコードを書いてきました。<br>
しかし、このテトリスなどのゲームや実際の製品でコードが数百〜数万行にまで膨れてくると、1つのファイルでは行数が多くて読みづらくなってしまいます。<br>
そこでPythonでは、複数のファイルにコードを分割することができます。<br>
実は今まで「おまじない」として使っていた`import`文の正体は、別のファイルにあるコードを使うための文法でした。<br><br>

試しに下のセルを実行してみてください。<br>
このセルにはクラスの定義文はおろか`print`文もないのに、いかにもクラスの関数を使用しているっぽい1行が書かれている上に、ある一文が出力されます。<br><br>

これは、同じにフォルダに保存された`import_tutorial.py`という別のファイルにあるコードを実行しているからです。<br>
`import_tutorial.py`には、文字を表示する機能である関数`printChar()`と、その文字を保存した変数`self.s`を定義する「コンストラクタ」を持つ、`ImportTutorial()`クラスが定義されています。<br>
クラスの持つ変数や機能を使うためにはクラスの「インスタンス」を作る必要があり、それは13行目で`IMPORT_TUTORIAL`という名前でなされています。<br><br>

テトリスの本番コードでも同じように、別のファイルにあるクラスが使われるときには<br>
`from (別のファイル名から拡張子「.py」を除いたもの) import (インスタンスを作った変数の名前)`<br>
とコードの冒頭に書かれていて、「別のファイルから変数を自分のコードに取り込できなさい」という命令を表しています。

In [1]:
from import_tutorial import IMPORT_TUTORIAL
IMPORT_TUTORIAL.printChar()

ここまで頑張りました!!Day4へようこそ〜


### 本番コードのファイル構成
複数ファイルにコードが分割されていても実行する方法があることを知った上で、テトリスの本番コードにおける`.py`ファイルの場所と役割を、<b><a href="https://github.com/seigot/tetris/tree/master#%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E6%A7%8B%E6%88%90">GitHubのREADMEファイル</a></b>で確認しておきましょう。<br><br>

テトリスを実行するときコマンドプロンプトやターミナルに<br>
`python start.py`<br>
という一行を入力して`Enter`キーを押していますが、これは「`start.py`の中身を実行せよ」という命令を意味していました。<br>
READMEの図は`start.py`を起点に、複数のファイルにまたがったクラスを呼び出している繋がりを表しています。<br><br>

上で述べた

- ミノの形・回転・移動・落下というルール

やゲーム画面の表示は、`game_manager`フォルダにある`game_manager.py`という<b>ゲーム管理用プログラム</b>と、`board_manager.py`という<b>ルール管理用プログラム</b>に、

- ミノをどの向きでどこに落とすか

という、皆さんに作っていただきたいミノの落とし方の作戦は、同じく`game_manager`フォルダにある`block_controller.py`、<br>
もしくは`game_manager`フォルダの中の`machne_learning`フォルダにある`block_controller_train.py`という<b>ブロック操作用プログラム</b>に、それぞれ実装されています。<br><br>

`block_controller.py`もしくは`block_controller_train.py`の中でも、<b>関数`GetNextMove()`が落とし方の作戦を計算している</b>部分です。<br>
本チュートリアルは、この関数で登場するミノの落とし方や作戦の計算方法を学ぶことが目的です。<br>
詳しい作りや流れはさておき、<span style="color: red; ">**実際にコードを読むときにはチュートリアル以上の内容は読み込まなくて問題ない**</span>ように、必要な部分だけを本番コードから抜き出して作っています。

<div align="center">
<img src="img/tetris_mermaid.png" width="500">
</div>

- - -

## 1. ミノの形と機能を定義しよう
・board_managerのShapeクラス<br>
・↑を辞書型GameStatusに格納するGame_managerのgetGameStatus


- - -

# 2. 定義したミノを動かしてみよう
・block_controllerのgetSearchXRangeらへん<br>
・対戦サーバのトレーニングページtest-4, test-5, tetris-1のイメージ

- - -

# 3. ミノを落とす方法を決めよう
・対戦サーバのトレーニングページtest-3のイメージ。あくまで「良さげ度合い」があることに触れるだけで、「良さげ度合い」の計算方法はDay6以降<br>
・辞書型nextMove

- - -

# 4. 【Day5のHint】ホールド機能を実装しよう
・exchangeHoldShapeに軽〜く触れる

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [2]:
colors = [[255,255,255],
          [255,0,0],
          [0,255,0],
          [255,255,0], 
          [218, 179, 0],
          [247, 171, 166],
          [0,0,255],
          [255,255,0]]

#ブロック可視化関数 (numpy 配列からテトリスの盤面を可視化)
def visualize_block(data, shape, size=0.5):
    block_array = []
    fig, ax = plt.subplots()
    fig.set_figwidth(shape[0] * size)
    fig.set_figheight(shape[1] * size)
    for i in range(shape[0]):
        row = []
        for j in range(shape[1]):
            c = colors[int(data[i][j])]
            row.append(c)
        block_array.append(row)
    block_array = np.array(block_array)
    im = plt.imshow(block_array)

    #グリッド線を引く
    ax.set_xticks(np.arange(-0.5, 10, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 23, 1.0), minor=True)
    ax.set_xticks(np.arange(-0.5, 10,10))
    ax.set_yticks(np.arange(-0.5, 23, 3))
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
    ax.grid(which='major', color='black', linestyle='-', linewidth=0.5)
    ax.imshow(block_array)
    
shape = tutorial_data["1"].shape 
visualize_block(tutorial_data["1"],shape) #サンプル用状態データ１を可視化


NameError: name 'tutorial_data' is not defined